# Read TileWall data and convert it to multple .json files


*I took the data from the google doc on the SVL drive, and converted it first to csv files.  Here I'm reading this in to create individual json files for each line.  I will write another code that will compile each individual json file into one json file to be read into the app.  I think this may be the easiest method for others to add additional objects.*

In [1]:
import pandas as pd
import numpy as np
import os
import wikipedia
import wptools #https://github.com/siznax/wptools

###  A function to grab an image from Wikipedia

*https://wikipedia.readthedocs.io/en/latest/quickstart.html*

In [2]:
def getWikiImages_org(nme, category):
    nme = nme.replace('(',' ')
    nme = nme.replace(')',' ')
    if (nme[0] == "M" and nme[1].isdigit()):
        nme = '"Messier '+nme[1:]+'"'
    if (category == "Galaxies"):
        category = "Galaxy"
    if (category == "Galaxy Clusters"):
        category = "Galaxy Cluster"
    if (category == "Nebulae"):
        category = "Nebula"
    if (category == "Star Clusters"):
        category = "Star Cluster"
    if (category == "Stars"):
        category = "Star"
    if (category == "Constellations"):
        category = "Constellation"
    search = nme + " " + '"'+category+'"' + " astronomy"
    site = wikipedia.search(search)
    print(search)
    if (site):
        print("wiki page: ", site[0])
        page = None
        try:
            page = wikipedia.page(site[0])
        except wikipedia.exceptions.DisambiguationError as e:
            #site = e.options
            pass
        if (page):
            img = []
            for i in page.images:
                if (("jpg" in i or "png" in i) and "Celestia" not in i):
                    img.append(i)
                    
                    #get more info with wptools
            #https://github.com/siznax/wptools/wiki/
            wpage = wptools.page(site[0])
            wpage.get_query()

            #get the main image
            mainImg = None
            if (wpage.images()):
                for i in wpage.images():
                    if (i['kind'] == 'query-pageimage'):
                        mainImg = i['url']

                if (mainImg):
                    img2 = img
                    img = [mainImg]
                    for i in img2:
                        if (i != mainImg):
                            img.append(i)
                        
            return img, page.url

    return None, None

*Updated below to match the ../addObject.ipynb code*

In [9]:
def getCaption(parsetree,img, maxLen = 1000, minLen = 10):
    p1 = 0
    if ('/' in img):
        p1 = img.rfind('/')+1
    imgSearch = img[p1:].replace('_',' ')
    print("\nSEARCHING FOR : ", img, imgSearch)

    #print("PARSETREE :",parsetree)
    caption = ''
    p1 = parsetree.lower().find(imgSearch.lower())
    if (p1 >= 0):
        caption = parsetree[p1+len(imgSearch):]
        #get the full caption, with all the markup
        left = 2
        right = 0
        for i,c in enumerate(caption):
            if (c == '['):
                left += 1
            if (c == ']'):
                right += 1
            if (left == right):
                break
        caption = caption[0:i-1]
        #print("first pass : ",caption)
        #remove the markup
        #for citations
        Ntrial = 0
        while ('{{' in caption and Ntrial < 100): 
            remove = []
            p1 = caption.find('{{')
            p2 = caption.find('}}')
            remove.extend([i+p1+1 for i in range(p2-p1)])
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
        
        #links to other wikipedia pages
        Ntrial = 0
        while ('[[' in caption and Ntrial < 100): 
            remove = []
            p1 = caption.find('[[')
            p2 = caption.find(']]')
            remove.extend([p1,p1+1, p2,p2+1])
            check = caption[p1:p2]
            #print("checking", check)
            p3 = check.find('|')
            if (p3 > 0):
                remove.extend([i+p1+1 for i in range(p3)])
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
            
        #other html markup
        Ntrial = 0
        done = False
        #print('caption to here : ', caption)
        while (not done and Ntrial < 100):
            remove = []
            p1 = caption.find('<ext>')
            p2 = caption.find('</ext>')
            if (p1 >=0 and p2 >=0):
                remove.extend([i+p1 for i in range(p2-p1+6)])
            else:
                done = True
                break
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
            
        #remove any bits at the beginning
        p1 = 0
        p = caption.rfind('|')
        if (p > 0):
            p1 = p
        caption = caption[p1+1:]
        
        
        if (len(caption) > maxLen or len(caption) < minLen):
            print("bad caption : ", len(caption))
            caption = None
            
    print("CAPTION : ", caption)
        
        
    return caption


def getWikiImages(nme, category):
    #https://wikipedia.readthedocs.io/en/latest/quickstart.html
    nme = nme.replace('(',' ')
    nme = nme.replace(')',' ')
    if (nme[0] == "M" and nme[1].isdigit()):
        nme = '"Messier '+nme[1:]+'"'
    if (category == "Galaxies"):
        category = "Galaxy"
    if (category == "Galaxy Clusters"):
        category = "Galaxy Cluster"
    if (category == "Nebulae"):
        category = "Nebula"
    if (category == "Star Clusters"):
        category = "Star Cluster"
    if (category == "Stars"):
        category = "Star"
    if (category == "Constellations"):
        category = "Constellation"
    search = nme + " " + '"'+category+'"' + " astronomy"
    site = wikipedia.search(search)    
    print(search)
    if (site):
        print("wiki page: ", site[0])
        page = None
        try:
            page = wikipedia.page(site[0])
        except wikipedia.exceptions.DisambiguationError as e:
            print(e.options)
            #pass
        if (page):
            images = []
            captions = []
            
            #get more info with wptools
            #https://github.com/siznax/wptools/wiki/
            wpage = wptools.page(site[0])
            wpage.get_more()
            wpage.get_query()
            wpage.get_parse()
            
            #get all the images (works better with wikipedia instead of wptools, but need wptools for captions)
            #print(page.images)
            #print(wpage.data['parsetree'])
            for i in page.images:
                if (("jpg" in i or "png" in i) and "Celestia" not in i):
                    cap = getCaption(wpage.data['parsetree'], i)
                    if (cap): #only keep images with captions.  I think this will cut down on the images we don't want.
                        captions.append(cap)
                        images.append(i)
                    
        
            #get the main image (only possible with wptools)
            mainImg = None
            mainCap = None
            if (wpage.images()):
                for i in wpage.images():
                    print('CHECKING :', i['kind'])
                    if (i['kind'] == 'query-pageimage'):
                        mainImg = i['url']
                        print('HAVE mainImg : ', mainImg)
                        if ('metadata' in i):
                            if ('ImageDescription' in i['metadata']):
                                mainCap = i['metadata']['ImageDescription']['value']
                                
                        if (mainCap == None):
                            mainCap = getCaption(wpage.data['parsetree'], i['url'])
                        print('HAVE mainCap :', mainCap)
                            
            if (mainImg):
                img2 = images
                caption2 = captions
                images = [mainImg]
                captions = [mainCap]
                for i,c in zip(img2,caption2):
                    if (i != mainImg):
                        images.append(i)
                        captions.append(c)
            

                
            return images, captions, page.url

    return None, None, None

In [10]:
files = ['TileWallGalaxies.csv', 'TileWallGalaxyClusters.csv','TileWallNebulae.csv', \
         'TileWallStarClusters.csv','TileWallStars.csv',  'TileWallConstellations.csv' ]
tpe = ['Galaxies', 'Galaxy Clusters', 'Nebulae', 'Star Clusters', 'Stars', 'Constellations']
#not sure what to do about the 'Constellation Detailed Notes'...

In [11]:
for i,f in enumerate(files):
    df = pd.read_csv(f, encoding = "ISO-8859-1")
    for index, row in df.iterrows():
        if (isinstance(row['Object'], str)):
            row['Category'] = tpe[i]
            d = {}
            nme = row['Object']
            fname = os.path.join('objectFiles',''.join(e for e in nme if e.isalnum())+'.json')
            print(fname)
            
            d[nme] = row
            d[nme].pop('Object')
            imgurls, captions, url = getWikiImages(nme, tpe[i])
            d[nme]['images'] = None
            d[nme]['captions'] = None
            d[nme]['wikipedia'] = None
            if (imgurls):
                d[nme]['images'] = imgurls
                d[nme]['captions'] = captions
                d[nme]['wikipedia'] = url
                print(imgurls[0])
            pd.DataFrame(d).to_json(fname)
            print("")

objectFiles/AndromedaM31.json
Andromeda M31  "Galaxy" astronomy
wiki page:  Andromeda Galaxy


en.wikipedia.org (querymore) Andromeda Galaxy
en.wikipedia.org (querymore) 74331 (&blcontinue=0|6390647)
en.wikipedia.org (querymore) 74331 (&blcontinue=3|38333229)
en.wikipedia.org (querymore) 74331 (&blcontinue=0|55515710)
Andromeda Galaxy (en) data
{
  backlinks: <list(1622)> {'pageid': 1925, 'ns': 0, 'title': 'Andr...
  categories: <list(12)> Category:Andromeda (constellation), Categ...
  contributors: 1,066
  files: <list(28)> File:A Swift Tour of M31.OGG, File:A Swift Tou...
  languages: <list(94)> {'lang': 'af', 'title': 'Andromeda-sterres...
  pageid: 74331
  requests: <list(4)> querymore, querymore, querymore, querymore
  title: Andromeda Galaxy
  views: 2,111
}
en.wikipedia.org (query) Andromeda Galaxy
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|NGC_147)
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|Pa...
en.wikipedia.org (imageinfo) File:Andromeda Galaxy (with h-alpha).jpg
Andromeda Galaxy (en) data
{
  WARNINGS: <dict(1)> extracts
  alias


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/9/92/Andromeda_active_core.jpg Andromeda active core.jpg
CAPTION :  Artist's concept of the Andromeda Galaxy core showing a view across a disk of young, blue stars encircling a supermassive black hole. NASA/ESA&amp;nbsp;photo.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/f/ff/Andromeda_galaxy.jpg Andromeda galaxy.jpg
CAPTION :  The Andromeda Galaxy pictured in ultraviolet light by GALEX.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/0/0d/Andromeda_galaxy_2.jpg Andromeda galaxy 2.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/a4/Charles_Messier.jpg Charles Messier.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/c/c5/Hubble_Finds_Giant_Halo_Around_the_Andromeda_Galaxy.jpg Hubble Finds Giant Halo Around the Andromeda Galaxy.jpg
CAPTION :  Giant halo around Andromeda Galaxy.

SEARCHING FOR :  https://upload.wikimedia.or

en.wikipedia.org (querymore) Antennae Galaxies
Antennae Galaxies (en) data
{
  backlinks: <list(362)> {'pageid': 11971, 'ns': 0, 'title': 'Gala...
  categories: <list(12)> Category:Arp objects, Category:Astronomic...
  contributors: 169
  files: <list(9)> File:Antennae Galaxies NGC4038 NGC4039 Goran Ni...
  languages: <list(35)> {'lang': 'af', 'title': 'Antenne-sterreste...
  pageid: 1111758
  requests: <list(1)> querymore
  title: Antennae Galaxies
  views: 112
}
en.wikipedia.org (query) Antennae Galaxies
en.wikipedia.org (query) Antennae Galaxies (&plcontinue=1111758|0...
en.wikipedia.org (imageinfo) File:Antennae Galaxies reloaded.jpg
Antennae Galaxies (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Astronomy
  backlinks: <list(362)> {'pageid': 11971, 'ns': 0, 'title': 'Gala...
  categories: <list(12)> Category:Arp objects, Category:Astronomic...
  contributors: 169
  description: interacting galaxies in the constellation Corvus
  extext: <str(344)> The **Antenna


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/a0/Antennae_Galaxies_NGC4038_NGC4039_Goran_Nilsson_%26_The_Liverpool_Telescop.jpg Antennae Galaxies NGC4038 NGC4039 Goran Nilsson %26 The Liverpool Telescop.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/0/03/Antennae_Galaxies_composite_of_ALMA_and_Hubble_observations.jpg Antennae Galaxies composite of ALMA and Hubble observations.jpg
bad caption :  2493
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/30/Antennae_Galaxies_reloaded.jpg Antennae Galaxies reloaded.jpg
bad caption :  5662
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/5/5d/VLT_Observes_the_Antennae_Galaxies.jpg VLT Observes the Antennae Galaxies.jpg
bad caption :  2493
CAPTION :  None
CHECKING query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/3/30/Antennae_Galaxies_reloaded.jpg
HAVE mainCap : The NASA/ESA Hubble Space Telescope

...


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/9/96/Pyramidi_aavikolla.png Pyramidi aavikolla.png
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/9/9e/Saqqarah_Djeser_10.jpg Saqqarah Djeser 10.jpg
CAPTION :  Djoser's ''ka'' statue in its serdab
https://upload.wikimedia.org/wikipedia/commons/9/9e/Saqqarah_Djeser_10.jpg

objectFiles/UrsaMinor.json
Ursa Minor "Constellation" astronomy
wiki page:  Ursa Minor


en.wikipedia.org (querymore) Ursa Minor
en.wikipedia.org (querymore) 31773 (&blcontinue=0|51145663)
Ursa Minor (en) data
{
  backlinks: <list(546)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories: <list(4)> Category:Constellations, Category:Constell...
  contributors: 633
  files: <list(9)> File:Blue pencil.svg, File:Celestia.png, File:C...
  languages: <list(92)> {'lang': 'af', 'title': 'Klein Beer'}, {'l...
  pageid: 31773
  requests: <list(2)> querymore, querymore
  title: Ursa Minor
  views: 472
}
en.wikipedia.org (query) Ursa Minor
en.wikipedia.org (imageinfo) File:Ursa Minor IAU.svg
Ursa Minor (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Little Bear, Little Dipper, UMi, Ursae Minoris
  assessments: <dict(1)> Astronomy
  backlinks: <list(546)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories: <list(4)> Category:Constellations, Category:Constell...
  contributors: 633
  description: <str(90)> constellation in the northern celestial h...
  extex


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/8/89/NGC_6217_hs-2009-25-bc-full_jpg.jpg NGC 6217 hs-2009-25-bc-full jpg.jpg
bad caption :  8
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/7/72/UrsaMinorCC.jpg UrsaMinorCC.jpg
CAPTION :  The constellation Ursa Minor as it can be seen by the naked eye (with connections and label added).

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/c/ce/Ursa_Major_-_Ursa_Minor_-_Polaris.jpg Ursa Major - Ursa Minor - Polaris.jpg
CAPTION :  Ursa Minor and Ursa Major in relation to Polaris

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/3f/Sidney_Hall_-_Urania%27s_Mirror_-_Draco_and_Ursa_Minor.jpg Sidney Hall - Urania%27s Mirror - Draco and Ursa Minor.jpg
CAPTION :  
CHECKING query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/c/cb/Ursa_Minor_IAU.svg
HAVE mainCap : IAU Ursa Minor chart
CHECKING query-thumbnail
https://upload.wikimedia.org/wiki

en.wikipedia.org (querymore) NGC 4567 and NGC 4568
NGC 4567 and NGC 4568 (en) data
{
  backlinks: <list(27)> {'pageid': 214067, 'ns': 0, 'title': 'Virg...
  categories: <list(7)> Category:Galaxy stubs, Category:Interactin...
  contributors: 68
  files: <list(4)> File:Artist’s impression of the Milky Way.jpg, ...
  languages: <list(13)> {'lang': 'ar', 'title': 'NGC 4567&NGC 4568...
  pageid: 4086864
  requests: <list(1)> querymore
  title: NGC 4567 and NGC 4568
  views: 12
}
en.wikipedia.org (query) NGC 4567 and NGC 4568
en.wikipedia.org (query) NGC 4567 and NGC 4568 (&plcontinue=40868...
en.wikipedia.org (imageinfo) File:NGC4568.jpg
NGC 4567 and NGC 4568 (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Siamese Twins, Butterfly Galaxies, KPG 347, VV 219
  assessments: <dict(1)> Astronomy
  backlinks: <list(27)> {'pageid': 214067, 'ns': 0, 'title': 'Virg...
  categories: <list(7)> Category:Galaxy stubs, Category:Interactin...
  contributors: 68
  description: galaxy pair i


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/4/4f/Artist%E2%80%99s_impression_of_the_Milky_Way.jpg Artist%E2%80%99s impression of the Milky Way.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/aa/NGC4567-ngc4568-hst-606.jpg NGC4567-ngc4568-hst-606.jpg
CAPTION :  NGC 4567 and 4568 imaged by the Hubble Space Telescope

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/ac/NGC4567_and_NGC4568_Goran_Nilsson_%26_The_Liverpool_Telescope.jpg NGC4567 and NGC4568 Goran Nilsson %26 The Liverpool Telescope.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/b/b5/NGC4568.jpg NGC4568.jpg
bad caption :  1440
CAPTION :  None
CHECKING query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/b/b5/NGC4568.jpg
HAVE mainCap : NGC 4567 (haut) et NGC 4568 (bas).
CHECKING query-thumbnail
CHECKING query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/b/b5/NGC4568.jpg